In [2]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from annoq_tree_gen import DTYPES
from base import load_json

tree_df = pd.read_csv("./../../annoq_data/input/annotation_tree.csv", sep=',', dtype=DTYPES)
#mappings = load_json("./../../annoq_data/mappings.json")

tree_df.set_index("id", inplace=True, drop=False)

In [3]:
tree_df = tree_df[~tree_df['name'].str.endswith('_list')]
tree_df

,id,parent_id,leaf,name,label,detail,link,pmid,sort,root_url,sample_url,field_type,keyword_searchable
id,,,,,,,,,,,,,
0,0,NaN,False,root,Annotation,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,1,0,False,Basic Info,NaN,"Basic information about the variant, such as c...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
26,26,0,False,ANNOVAR,NaN,Pre-computed ANNOVAR annotations for all alter...,http://annovar.openbioinformatics.org/en/lates...,20601685,2.0,NaN,NaN,NaN,NaN
208,208,0,False,SnpEff,NaN,AnpEff is a program for annotating and predict...,http://pcingola.github.io/SnpEff/,22728672,3.0,NaN,NaN,NaN,NaN
132,132,0,False,VEP,NaN,Variant Effect Predictor (VEP) is developed by...,https://uswest.ensembl.org/info/docs/tools/vep...,27268795,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,621,495,True,sno_miRNA_type,NaN,the type of snoRNA or miRNA (from miRBase/snoR...,NaN,NaN,NaN,NaN,NaN,text,True
622,622,495,True,splicing_consensus_ada_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,NaN,NaN,float,NaN
623,623,495,True,splicing_consensus_rf_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,NaN,NaN,float,NaN


In [4]:
def is_leaf(G, node):
    return G.out_degree(node) == 0 and G.in_degree(node) == 1

def add_value_type(name):
    if '_list_id' in name:
        return 'term_id'
        
    return None



tree_df['value_type'] = tree_df.apply(lambda row: add_value_type(row['name']),axis=1)
#tree_df[tree_df['value_type']=='term_id']
tree_df

,id,parent_id,leaf,name,label,detail,link,pmid,sort,root_url,sample_url,field_type,keyword_searchable,value_type
id,,,,,,,,,,,,,,
0,0,NaN,False,root,Annotation,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
1,1,0,False,Basic Info,NaN,"Basic information about the variant, such as c...",NaN,NaN,1.0,NaN,NaN,NaN,NaN,None
26,26,0,False,ANNOVAR,NaN,Pre-computed ANNOVAR annotations for all alter...,http://annovar.openbioinformatics.org/en/lates...,20601685,2.0,NaN,NaN,NaN,NaN,None
208,208,0,False,SnpEff,NaN,AnpEff is a program for annotating and predict...,http://pcingola.github.io/SnpEff/,22728672,3.0,NaN,NaN,NaN,NaN,None
132,132,0,False,VEP,NaN,Variant Effect Predictor (VEP) is developed by...,https://uswest.ensembl.org/info/docs/tools/vep...,27268795,4.0,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,621,495,True,sno_miRNA_type,NaN,the type of snoRNA or miRNA (from miRBase/snoR...,NaN,NaN,NaN,NaN,NaN,text,True,None
622,622,495,True,splicing_consensus_ada_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,NaN,NaN,float,NaN,None
623,623,495,True,splicing_consensus_rf_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,NaN,NaN,float,NaN,None


In [5]:
g = nx.from_pandas_edgelist(
    tree_df, 'parent_id', 'id', create_using=nx.DiGraph())

roots = [tree_df.loc[n]['name'] for n in nx.descendants(g,'26') if is_leaf(g, n)]

#len(roots)

def leaf_count(G, source):
    children = [n for n in nx.descendants(G, source) if is_leaf(g, n)]
    return len(children)


def descendants_count(G, source):
    children = [n for n in nx.descendants(G, source)]
    return len(children)

def parents_count(G, source):
    children = [n for n in nx.descendants(G, source) if not is_leaf(g, n)]
    return len(children)

        
    
#get_type(mappings, 'ref')
#child_count(g, '26')
#nx.descendants(g,'26')

In [7]:
tree_df['children_count'] = tree_df.apply(lambda row: descendants_count(g, row['id']),axis=1)
tree_df['category_count'] = tree_df.apply(lambda row: parents_count(g, row['id']),axis=1)
tree_df['leaf_count'] = tree_df.apply(lambda row: leaf_count(g, row['id']),axis=1)
tree_df

,id,parent_id,leaf,name,label,detail,link,pmid,sort,root_url,sample_url,field_type,keyword_searchable,value_type,children_count,category_count,leaf_count
id,,,,,,,,,,,,,,,,,
0,0,NaN,False,root,Annotation,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,545,18,527
1,1,0,False,Basic Info,NaN,"Basic information about the variant, such as c...",NaN,NaN,1.0,NaN,NaN,NaN,NaN,None,5,0,5
26,26,0,False,ANNOVAR,NaN,Pre-computed ANNOVAR annotations for all alter...,http://annovar.openbioinformatics.org/en/lates...,20601685,2.0,NaN,NaN,NaN,NaN,None,40,0,40
208,208,0,False,SnpEff,NaN,AnpEff is a program for annotating and predict...,http://pcingola.github.io/SnpEff/,22728672,3.0,NaN,NaN,NaN,NaN,None,63,0,63
132,132,0,False,VEP,NaN,Variant Effect Predictor (VEP) is developed by...,https://uswest.ensembl.org/info/docs/tools/vep...,27268795,4.0,NaN,NaN,NaN,NaN,None,59,0,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,621,495,True,sno_miRNA_type,NaN,the type of snoRNA or miRNA (from miRBase/snoR...,NaN,NaN,NaN,NaN,NaN,text,True,None,0,0,0
622,622,495,True,splicing_consensus_ada_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,NaN,NaN,float,NaN,None,0,0,0
623,623,495,True,splicing_consensus_rf_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,NaN,NaN,float,NaN,None,0,0,0


In [ ]:
annotations.to_csv('./../../annoq-data/tree-with-types.csv', index = False)